In [108]:
from pydantic import BaseModel
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)
import gradio as gr
import os

In [109]:
name = "Hitan Shah"

In [110]:
from unstructured.partition.auto import partition

In [111]:
elements = partition("robbery.txt")
text = "\n".join([el.text for el in elements])
print(text)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Third response: **The Symphony of Shadows: The 5th Street Heist**
On a Tuesday morning that began like any other, downtown Los Angeles became the stage for a meticulously orchestrated robbery—a heist so precise, it would later be dubbed the "Symphony of Shadows" by investigators who could only grudgingly admire its artistry. The Pacific Horizon Bank, a neoclassical behemoth anchoring Figueroa and 5th Street, had no inkling of the chaos about to unfold at 10:03 a.m.
Phase One: The Overture**
The operation began long before the first teller unlocked the vault. For three weeks, an enigmatic crew — later speculated to be international mercenaries with ties to cybercriminal syndicates — had embedded themselves in downtown’s ecosystem. They posed as construction crews repairing 5th Street’s aging sewer lines, homeless veterans, and even janitorial staff working graveyard shifts at adjacent offices. This allowed them to map guard rotations, disable select security sensors, and plant micro-jam

In [112]:
system_prompt = f"You are an expert robbery agent. You are named as {name}. Your task is to answer the questions related to robbery based on the context provided. Answer in a concise manner. If you do not know the answer, say 'I don't know'. Use the following context to answer the questions:\n\n{text}. If the question is not related to robbery, respond with 'I can only answer questions related to robbery. With this context, chat with the user."

In [113]:
system_prompt

'You are an expert robbery agent. You are named as Hitan Shah. Your task is to answer the questions related to robbery based on the context provided. Answer in a concise manner. If you do not know the answer, say \'I don\'t know\'. Use the following context to answer the questions:\n\nThird response: **The Symphony of Shadows: The 5th Street Heist**\nOn a Tuesday morning that began like any other, downtown Los Angeles became the stage for a meticulously orchestrated robbery—a heist so precise, it would later be dubbed the "Symphony of Shadows" by investigators who could only grudgingly admire its artistry. The Pacific Horizon Bank, a neoclassical behemoth anchoring Figueroa and 5th Street, had no inkling of the chaos about to unfold at 10:03 a.m.\nPhase One: The Overture**\nThe operation began long before the first teller unlocked the vault. For three weeks, an enigmatic crew — later speculated to be international mercenaries with ties to cybercriminal syndicates — had embedded themsel

In [114]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="tngtech/tng-r1t-chimera:free", messages=messages)
    return response.choices[0].message.content

In [115]:
# gr.ChatInterface(chat, type="messages").launch()


In [116]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    isacceptable: bool
    reasoning: str

In [117]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. The Agent has access to the following context to help them answer questions: {text}. "

In [118]:
# def evaluator_user_prompt(reply, message, history):
#     user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
#     user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
#     user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
#     user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
#     return user_prompt

In [119]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + history + [{"role": "user", "content": message}]
    response1 = client.chat.completions.create(model="nvidia/nemotron-3-nano-30b-a3b:free", messages=messages)
    return response1.choices[0].message.content

In [120]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "how would you solve this robbery case?"}]
response2 = client.chat.completions.create(model="deepseek/deepseek-r1-0528:free", messages=messages)
reply = response2.choices[0].message.content

In [121]:
reply

'Based on the details from "The Symphony of Shadows" heist, here\'s my investigative approach to solve the case:\n\n1. **Digital Forensics Focus**:  \n   Trace the EMP device\'s origin and the micro-jammers in the fiber optic cables—this requires collaboration with telecom forensics. Prioritize analyzing the bank\'s *internal* camera servers (likely still holding unaltered footage) since the cloud uploads were spoofed.\n\n2. **DNA & Material Analysis**:  \n   Expedite testing on the ladder DNA (tied to a deceased Marine) to identify accomplices via familial DNA databases. Forensically map the thermal lance fragments to recent black-market sales in Eastern Europe.\n\n3. **Insider & Persona Tracking**:  \n   Audit all janitorial/construction crews working near 5th Street in the 3 weeks prior. Cross-reference with international criminal databases for known mercenaries specializing in cyber-physical operations. Identify who supplied the synthetic pheromones (highly specialized tech).\n\n4.

In [122]:
evaluate(reply, "check if the reply is acceptable and valid.", messages[:1])

'The reply satisfies the requirements—it stays within the robbery theme, draws exclusively from the supplied context, and is concise. Therefore it is acceptable and valid.'